In [41]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist

# 모델 생성 함수
def get_mnist_model():
    model = Sequential([
        layers.Flatten(input_shape=(28 * 28,)),  # 입력을 784 차원으로 설정
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# 모델 초기화
model = get_mnist_model()
loss_fn = SparseCategoricalCrossentropy()
optimizer = RMSprop()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

# 훈련 함수 정의
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

# 메트릭 초기화 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

# 데이터 로드 및 전처리
(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

# 데이터셋 준비
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(32)

# 훈련 루프
epochs = 3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch + 1}번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

# 단계별 평가 루프
@tf.function
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs['val_'+metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs['val_loss'] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print('평가 결과:')
for key, value in logs.items():
    print(f'...{key}: {value:.4f}')

c:\Users\envym\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1번째 에포크 결과
...sparse_categorical_accuracy: 0.9263
...loss: 0.2510
2번째 에포크 결과
...sparse_categorical_accuracy: 0.9670
...loss: 0.1119
3번째 에포크 결과
...sparse_categorical_accuracy: 0.9771
...loss: 0.0807
평가 결과:
...val_sparse_categorical_accuracy: 0.9657
...val_loss: 0.1212


In [ ]:
#fit()이 사용할 사용자 정의 훈련 스텝 구현
loss_fn = SparseCategoricalCrossentropy()
loss_tracker = metrics.Mean(name='loss')

class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        loss_tracker.update_state(loss)
        return {'loss': loss_tracker.result()}
    
    @property
    def metrics(self):
        return [loss_tracker]

# 모델 정의
inputs = Input(shape=(28 * 28,))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)
model = CustomModel(inputs, outputs)

# 컴파일 및 훈련
model.compile(optimizer=optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.4561
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1646
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1352


In [61]:
# compile() 메소드를 통해 지표와 손실 설정
from tensorflow.keras.metrics import SparseCategoricalAccuracy
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.compiled_metrics.update_state(targets, predictions)

        
        return {m.name: m.result() for m in self.metrics}

# 모델 정의
inputs = Input(shape=(28 * 28,))
features = layers.Dense(512, activation='relu')(inputs)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(10, activation='softmax')(features)
model = CustomModel(inputs, outputs)

# 컴파일 및 훈련
model.compile(
    optimizer=optimizers.RMSprop(),
    loss=SparseCategoricalCrossentropy(),  # 손실 함수 설정
    metrics=[SparseCategoricalAccuracy()]   # 정확도 메트릭 설정
)
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - sparse_categorical_accuracy: 0.8655 - loss: 0.1000
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - sparse_categorical_accuracy: 0.9516 - loss: 0.1000
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - sparse_categorical_accuracy: 0.9620 - loss: 0.1000


# 점점 속도가 빨라진다.
## 자꾸 해보니 모델 구조가 조금씩 눈에 익어간다.
### 모델이 뭘 학습하고 뱉어내는지 감만 잡힌다.